# <div class="alert alert-danger" > Tracking Objects in Video with Particle Filters


### <div class="alert alert-success" >Import libraries

In [42]:
import numpy as np
import cv2

# Repeatability
np.random.seed(0)

VFILENAME = "walking.mp4"
HEIGHT = 406
WIDTH = 722

### <div class="alert alert-success" >Load video frames from file

In [43]:
def get_frames(filename): 
    video = cv2.VideoCapture(filename)  # loading the video in the video object
    while video.isOpened():             
        ret, frame = video.read()      # ret is a flag 
        if ret:                        # if ret true yeild frame else brreak
            yield frame
        else:
            break
    video.release()
    yield None

### <div class="alert alert-success" >Creating a particle cloud

In [44]:
NUM_PARTICLES = 5000 # 50
VEL_RANGE = 0.5
def initialize_particles():
    
    particles = np.random.rand(NUM_PARTICLES, 4)
    
    particles = particles * np.array( (WIDTH,HEIGHT,VEL_RANGE,VEL_RANGE) )
    
    # decriment particles to VEL_RANGE/2.0
    
    particles[ :, 2:4 ] -= VEL_RANGE/2.0 # Center velocities around 0 so it can go to positive and negative
    
#     # printing some of these particles on the video
#     print(particles[ :, 20: ])
    
    
    return particles

### <div class="alert alert-success" >Moving particles according to their velocity state

In [45]:
def apply_velocity(particles):
    particles[ :, 0 ] += particles[ :, 2 ]  # x = x + u
    particles[ :, 1 ] += particles[ :, 3 ]
    return particles

### <div class="alert alert-success" >Prevent particles from falling off the edge of the video frame
    
<b>particles[i,0] = min(WIDTH-1, particles[i,0])</b> will prevent x value from getting grater then WIDTH-1. we subtract 1 because the frame coordinate is zero based so if you have a frame  with 100 pixels width you would want the coordinate to go from 0 to 99.
    
<b>particles[i,0] = max(0, min(WIDTH-1, particles[i,0]))</b> This will prevent it from going to and below zero
    

In [46]:
def enforce_edges(particles):
    
    for i in range(NUM_PARTICLES):
        
        particles[i,0] = max(0, min(WIDTH-1, particles[i,0])) 
        particles[i,1] = max(0, min(HEIGHT-1, particles[i,1]))
        
    return particles

### <div class="alert alert-success" >Measure each particle's quality
    
In order to improve our state estimation, we want
to check the color of the pixel sitting under each particle
and compare it to the target color.
So now, to get the precise values of that target color,
I paused the video just like this, and I took a screenshot
and I opened it in gimp and with the eyedropper tool.
I found the BGR values for some pixel.
So I just chose a pixel on the sleeve of our target's
vest here.
Try to choose a pixel
that's kind of representative of the whole target.
Not too bright, not too dark.
And I ended up with values 156, 74, 38.
So let's store those values in a numpy array. 
Again, 156, 74, and 38.
Now I'll create a numpy array to store those color
differences. We'll call it errors, and we'll fill it with zeros to start with.
It will have one element for each particle.
    

In [47]:
def compute_errors(particles, frame):
    errors = np.zeros(NUM_PARTICLES)
    TARGET_COLOUR = np.array( (189,105,82) ) # Blue top sleeve pixel colour
#    TARGET_COLOUR = np.array( (148, 73, 49) ) # Blue top sleeve pixel colour
    
    for i in range(NUM_PARTICLES):
        
        x = int(particles[i,0])
        y = int(particles[i,1])
        
        pixel_colour = frame[ y, x, : ]
        
        errors[i] = np.sum( ( TARGET_COLOUR - pixel_colour )**2 ) # Mean Square Error in colour space
    return errors

### <div class="alert alert-success" >Assign weights to the particles based on their quality of match
    
    
 we will compute particle weights and re-sample
the particle filter.
The next step is to use the errors we calculated to compute
a weight for each particle.
When the error is low, we want the weight to be high.
This means that a particle is at a location where the pixel
color is a good match for the target.
The simplest thing we can do is invert the errors,
In this sense: if we take the highest error that was found
and then we subtract off the errors array, this will be done
element-wise, and the weights array will then have as many
elements as the errors array has.
And next we want to prevent the particles from piling up
along the edge.
So we'd like to set the weight for particles on the edge
to zero.
We can use a bit of really nice numpy syntax to do this
in one line.  So we know that we want to set some elements
of the weights array to zero. So here's the trick.
First of all, if you use a numpy array in a conditional
expression... so this is all the rows and the column
that's zero.
What that expression will return is a new numpy array
with Trues or Falses depending on how it evaluated element-wise,
and we can use that array of Trues and Falses
as an index for weights.
So this is the condition, x is equal to zero.
That's along the left hand edge of the frame.
We can do a logical OR and set the condition where,
if the x particle value is equal to WIDTH - 1.
So this would be the right edge of the frame, and we do
the same thing for y, so y will be zero at the top edge
of the frame and at the bottom edge, it will be equal
to HEIGHT-1.
So there you have it in one expression.


In [48]:
def compute_weights(errors):
    weights = np.max(errors) - errors
    weights[ 
        (particles[ :,0 ] == 0) |        # left edge of the frame
        (particles[ :,0 ] == WIDTH-1) |  # Right edge of the frame
        (particles[ :,1 ] == 0) |        # top edge of the frame
        (particles[ :,1 ] == HEIGHT-1)   # bottom edge of the frame
    ] = 0.0
    
    # Make weights more sensitive to colour difference.
    # Cubing a set of numbers in the interval [0,1], the farther a number is from 1, the more it gets squashed toward zero
    weights = weights**4
    
    return weights

### <div class="alert alert-success" >Resample particles according to their weights
    
Now we're going to do something really cool with these weights.
If we normalize them so that they sum to one, we can use them
as a probability distribution over the particles.
Next, we want to re-sample the particles according
to these probabilities.
So in other words, we're going to build a new particle array
by sampling from the current particles.  The ones that have
a high weight will get chosen many times and those with a 
low weight may not be chosen at all.
numpy has a very useful function for this kind of re sampling.
We're going to call the choice function.
And here are the arguments we hand it.
First of all, we need to tell it what its sampling from.
So if we just give it a single integer, it's clever enough
to know that we want to re-sample in the range from zero
to that maximum value.
Next we tell it, how many samples to take?
So since we're replacing the whole particle array, we need
as many samples as we have particles.
And finally, we just tell it the probability distribution.
So once we're done with this re sampling, we have our new
index numbers that are pointing to the current particles
we've sampled.
Then we just rebuild a particle array according
to these index numbers.  Right.
So these are pointing to the rows of the previous particle
array. So finally, it would be great if we could come up
with a single x,y position, which is our best guess for where
is the target.
So far, we've expressed our state estimation using all
the particles.
But we'd like to have a single best guess, and we can get
that very easily by just taking the average x and y position
over all the particles that we have.
So x will be mean of all the x values and likewise for y,
the mean over all the y values.
So finally, at the end, we are returning the particles array
and we'd like to return at tuple with the x and y cast
to integers.
So this will allow us to use this tuple directly to index
as pixel coordinates.


In [49]:
def resample(particles, weights):
    # Normalize to get valid PDF
    probabilities = weights / np.sum(weights)

    # Resample
    indices = np.random.choice(
        NUM_PARTICLES,         # from where to take samples
        size=NUM_PARTICLES,    # number of samples to take
        p=probabilities)      # tell it the probability distribution
    
    particles = particles[ indices, : ]

    # Take average over all particles, best-guess for location
    x = np.mean(particles[:,0])
    y = np.mean(particles[:,1])
    return particles, (int(x),int(y))

### <div class="alert alert-success" >Fuzz the particles
    
So let's start by cleaning up the damage we had from last
session.
We can do a Kernel - Restart Kernel, and that will close
down the window and the resources that were being held
by the crashed program.
So in the previous task, our particle filter did a great job
of converging on one pixel.
But it wasn't quite a pixel on the target.
We need it to locate the target and keep tracking the target,
even if it moves around the frame or the lighting conditions
change.
The solution, believe it or not, is to add noise.
As an engineer, I have always thought that noise
in any system was a bad thing.
But in a particle filter, we can use noise to express our
uncertainty about the target state.
We will generate some Gaussian noise and add it
to each particle.
If the target changes in the next frame, some
of the particles will have changed in the same way, thanks
to the variations from the noise we added, so they will move
along with the target.
The other particles that did not move with the target
will have more color error and won't get re-sampled.
So let's code this.  We're using Gaussian noise,
so we're going to specify the standard deviations
for position.
We can go with a standard standard deviation of one pixel
and for velocity, maybe half a pixel per frame.
So next we create our noise: we're going to create it one column
at a time, and then we'll concatenate all the columns
into one array.
Okay, so here we go.
The first column is for x position.  We'll generate some Gaussian
noise according to the position sigma,
the standard deviation we specified.  And the size
of the output noise array here.
It's going to be like just a single column.
So NUM_PARTICLES is the number of rows and one column.
Okay, so you want to do the same thing for y it's exactly
the same line, just Copy-Paste.  And paste it two more times
for velocity.
We're just going to use a different standard deviation,
the one for velocity.
And finally, we need to tell numpy how to concatenate all
these things.
So we want it to coordinate column-wise, specify that
with axis=1.
Finally, now that we have a noise array, we just increment
the particles array by the noise and that's it.


In [50]:
def apply_noise(particles):
    # Noise is good!  Noise expresses our uncertainty in the target's position and velocity
    # We add small variations to each hypothesis that were samples from the best ones in last iteration.
    # The target's position and velocity may have changed since the last frame, some of the fuzzed hypotheses will match these changes.
   
    POS_SIGMA = 1.0   # standard deviation of i pixel
    VEL_SIGMA = 0.5   # velocity of 0.5
    
    noise = np.concatenate(
        (
            np.random.normal(0.0, POS_SIGMA, (NUM_PARTICLES,1)),
            np.random.normal(0.0, POS_SIGMA, (NUM_PARTICLES,1)),
            np.random.normal(0.0, VEL_SIGMA, (NUM_PARTICLES,1)),
            np.random.normal(0.0, VEL_SIGMA, (NUM_PARTICLES,1))
        ),
        axis=1    # concatenate column wise
    )
    particles += noise
    return particles

### <div class="alert alert-success" >Display the video frames

In [51]:
def display(frame, particles, location):
    
    if len(particles) > 0:        # checking if there are any particals to display
        
        for i in range(NUM_PARTICLES):
            x = int(particles[i,0])    # using partical values as pixel coordinates
            y = int(particles[i,1])    # using partical values as pixel coordinates
            
#            cv2.circle(frame, (x,y), 1, (0,255,0), 1)   # color convention is (blue,green,red)
    if len(location) > 0:
        cv2.circle(frame, location, 15, (0,0,255), 5)
    cv2.imshow('frame', frame)
    
    # Means for pausing the video playback
    
    if cv2.waitKey(30) == 27: # wait 30 msec for user to his Esc key (27) 
        if cv2.waitKey(0) == 27: # second Esc key exits program
            return True
    return False

### <div class="alert alert-success" >Main routine

In [52]:
particles = initialize_particles()

for frame in get_frames(VFILENAME):
    if frame is None: break

    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    errors = compute_errors(particles, frame)
    weights = compute_weights(errors)
    particles, location = resample(particles, weights)
    particles = apply_noise(particles)
    terminate = display(frame, particles, location)
    if terminate:
        break
cv2.destroyAllWindows()